In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import requests
from bs4 import BeautifulSoup

from time import sleep

Scroll to the bottom of the page until search for more button shows up

In [36]:
service = Service(r"C:\Users\Lenovo\Desktop\chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get('https://www.google.com/')

search = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search.send_keys('giraffe')
search.send_keys(Keys.ENTER)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a')))

image = driver.find_element(By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a')
image.click()

driver.execute_script('return document.body.scrollHeight')

for i in range(5):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    sleep(3)

# IMDB - Top 100 Greatest Movies of all Time

Fetch relevant data about the top 50 greatest movies of all time from IMDB

In [57]:
service = Service(r"C:\Users\Lenovo\Desktop\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get('https://www.google.com/')

search = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search.send_keys('top 100 greatest movies of all time')
search.send_keys(Keys.ENTER)

imdb_path = '//*[@id="rso"]/div[1]/block-component/div/div[1]/div/div/div/div/div[1]/div/div/div/div/div/div[2]/div/div/div[1]/a/h3'
driver.find_element(By.XPATH, imdb_path).click()

sleep(5)

names = []
descs = []
scores = []
grosses = []

for i in range(1, 101):
    sleep(0.5)
    name_path = '//*[@id="main"]/div/div[4]/div[3]/div[' + str(i) + ']/div[2]/h3/a'
    name = driver.find_element(By.XPATH, name_path).text
    names.append(name)
    
    desc_path = '//*[@id="main"]/div/div[4]/div[3]/div[' + str(i) + ']/div[2]/p[2]'
    desc = driver.find_element(By.XPATH, desc_path).text
    descs.append(desc)
    
    score_path = '//*[@id="main"]/div/div[4]/div[3]/div[' + str(i) + ']/div[2]/div[1]/div[1]/span[2]'
    score = driver.find_element(By.XPATH, score_path).text
    scores.append(score)
    
    try:
        gross_path = '//*[@id="main"]/div/div[4]/div[3]/div[' + str(i) + ']/div[2]/p[4]/span[5]'
        gross = driver.find_element(By.XPATH, gross_path).text
        grosses.append(gross)
    except NoSuchElementException:
        grosses.append('not found')

In [58]:
len(names)

100

In [59]:
scores = [float(score.replace(',', '.')) for score in scores]

In [60]:
import pandas as pd 

df = pd.DataFrame({'name':names, 'description':descs, 'gross':grosses, 'score':scores})

In [61]:
df

,name,description,gross,score
0,Baba,The aging patriarch of an organized crime dyna...,$134.97M,9.2
1,Esaretin Bedeli,Two imprisoned men bond over a number of years...,$28.34M,9.3
2,Schindler'in Listesi,"In German-occupied Poland during World War II,...",$96.90M,9.0
3,Kızgın Boğa,"The life of boxer Jake LaMotta, whose violence...",$23.38M,8.2
4,Casablanca,A cynical expatriate American cafe owner strug...,$1.02M,8.5
...,...,...,...,...
95,Arka Pencere,A wheelchair-bound photographer spies on his n...,$36.76M,8.5
96,Üçüncü Adam Kim?,Pulp novelist Holly Martins travels to shadowy...,$0.45M,8.1
97,Asi gençlik,A rebellious young man with a troubled past co...,not found,7.6
98,Gizli Teşkilât,A New York City advertising executive goes on ...,$13.28M,8.3


# Infinite Scrolling

Get the name and prices of nike sneakers through infinite scrolling

In [84]:
service = Service(r"C:\Users\Lenovo\Desktop\chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get('https://www.nike.com/tr/w/erkek-ayakkabilar-nik1zy7ok')

start = 0
new_height = driver.execute_script('return document.body.scrollHeight')
while True:
    if start == new_height:
        print(f'* The last height is {last_height}\ The end of the page')
        break
    else:    
        driver.execute_script(f'window.scrollTo({start}, {new_height})')
        sleep(2)
        start = new_height
        new_height = driver.execute_script('return document.body.scrollHeight')
        
soup = BeautifulSoup(driver.page_source, 'lxml')

names = []
prices = []
for item in soup.find_all(class_='product-card__link-overlay'):
    names.append(item.text)
    
for item in soup.find_all(class_='product-price css-11s12ax is--current-price'):
    prices.append(item.text)


* The last height is 69251\ The end of the page


In [89]:
df = pd.DataFrame({'name':names, 'price':prices})
df

,name,price
0,Nike Air Max Plus,"₺3.049,90"
1,Nike Cortez BE TRUE,"₺1.569,90"
2,Nike Metcon 7 AMP,"₺2.299,90"
3,Nike SuperRep Go 3 Next Nature Flyknit,"₺1.649,90"
4,Nike Air Zoom Pegasus 39,"₺1.979,90"
...,...,...
259,Nike Air Zoom Alphafly NEXT% Flyknit Ekiden,"₺5.129,90"
260,Nike Phantom GT2 Elite SG-Pro AC,"₺3.769,90"
261,Nike Air Max 95 EC,"₺3.049,90"
262,Nike Air Max 97 Unlocked By Vinnie,"₺3.799,90"


# Children's Fabric

Fetch the relevant data about children's fabric from https://www.fabric.com/quilting-fabric-children-39-s-cotton-fabrics.aspx

In [18]:
service = Service(r"C:\Users\Lenovo\Desktop\chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get('https://www.fabric.com/quilting-fabric-children-39-s-cotton-fabrics.aspx')

names = []
brands = []
contents = []
weights = []
types = []
descs = []
for i in range(1, 73):
    if i < 10:
        i = '0' + str(i)
        print(i)
        
    product_path = '//*[@id="rptProducts_ctl' + str(i) + '_ctlProductSimple_hlProductName"]'
    
    name = driver.find_element(By.XPATH, product_path).text
    print(name)
    names.append(name)
    
    link = driver.find_element(By.XPATH, product_path).get_attribute('href')
    print(link)
    driver.get(link)
    
    try:
        brand = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[3]/section[1]/div/div[2]/ul/li/a').text
        brands.append(brand)
    except NoSuchElementException:
        brands.append('not found')
        
    try:
        content = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[3]/section[1]/div/div[1]/table/tbody/tr[2]/td').text
        contents.append(content)
    except NoSuchElementException:
        contents.append('not found')    
    
    try:
        type = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[3]/section[1]/div/div[1]/table/tbody/tr[5]/td/ul/li').text
        types.append(type)
    except NoSuchElementException:
        types.append('not found') 
        
    try:
        weight = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[3]/section[1]/div/div[1]/table/tbody/tr[3]/td').text
        weights.append(weight)
    except NoSuchElementException:
        weights.append('not found')     
    
    
    try:
        desc = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[3]/section[1]/div/div[2]/div').text
        descs.append(desc)
    except NoSuchElementException:
        descs.append('not found')

    driver.back()
    

01
Comfy Flannel Print Alphabet & Motifs White
https://www.fabric.com/buy/0764989/comfy-flannel-print-alphabet-motifs-white
02
Charms Bees Yellow
https://www.fabric.com/buy/cw-843/charms-bees-yellow
03
Timeless Treasures Little Star Flannel Bunnies And Little Ones With Moons Grey
https://www.fabric.com/buy/0581907/timeless-treasures-little-star-flannel-bunnies-and-little-ones-with-moons-grey
04
Timeless Treasures You Are My Sunshine Bees Black
https://www.fabric.com/buy/0538306/timeless-treasures-you-are-my-sunshine-bees-black
05
The Kids are Alright Sheep Pink
https://www.fabric.com/buy/0742004/the-kids-are-alright-sheep-pink
06
Animal Friends Soft Book Panel Cream
https://www.fabric.com/buy/0386449/animal-friends-soft-book-panel-cream
07
Polka Dot Red
https://www.fabric.com/buy/0650276/polka-dot-red
08
Disney Ultimate Princess Celebration Princess Icon Packed Multi
https://www.fabric.com/buy/0803736/disney-ultimate-princess-celebration-princess-icon-packed-multi
09
Disney Pooh Nurser

Get all the images corresponding to a product

In [20]:
text = requests.get('https://www.fabric.com/quilting-fabric-children-39-s-cotton-fabrics.aspx').text
soup = BeautifulSoup(text, 'lxml')

images = []
for item in soup.find_all('img'):
    images.append(item.get('src'))    

In [30]:
images[14:30]

['https://images.fabric.com/images/279/279/0764989.jpg',
 'https://images.fabric.com/images/279/279/CW-843.jpg',
 'https://images.fabric.com/images/279/279/0581907.jpg',
 'https://images.fabric.com/images/279/279/0538306.jpg',
 'https://images.fabric.com/images/279/279/0742004.jpg',
 'https://images.fabric.com/images/279/279/0386449.jpg',
 'https://images.fabric.com/images/279/279/0650276.jpg',
 'https://images.fabric.com/images/279/279/0803736.jpg',
 'https://images.fabric.com/images/279/279/0681349.jpg',
 'https://images.fabric.com/images/279/279/0579344.jpg',
 'https://images.fabric.com/images/279/279/0725068.jpg',
 'https://images.fabric.com/images/279/279/0691269.jpg',
 'https://images.fabric.com/images/279/279/0581897.jpg',
 'https://images.fabric.com/images/279/279/BI-250.jpg',
 'https://images.fabric.com/images/279/279/0561200.jpg',
 'https://images.fabric.com/images/279/279/0402663.jpg']

In [36]:
df = pd.DataFrame({'name':names,'brand':brands, 'content':contents, 'weight':weights, 'type':types, 'image':images[14:86] })

In [37]:
df

,name,brand,content,weight,type,image
0,Comfy Flannel Print Alphabet & Motifs White,A.E. Nathan,100% Cotton,Lightweight (4.82 oz/yd²),Flannel,https://images.fabric.com/images/279/279/07649...
1,Charms Bees Yellow,Fabric Traditions,100% cotton,Lightweight (4.32 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/CW-84...
2,Timeless Treasures Little Star Flannel Bunnies...,Timeless Treasures,100% Cotton,Lightweight (4.32 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/05819...
3,Timeless Treasures You Are My Sunshine Bees Black,Timeless Treasures,100% Cotton,Lightweight (4.32 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/05383...
4,The Kids are Alright Sheep Pink,Santee Print Works,100% Cotton,Lightweight (3.84 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/07420...
...,...,...,...,...,...,...
67,Disney Pooh Nursery Pooh And Friends Tossed Multi,Disney,100% Cotton,Lightweight (4.45 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/07494...
68,In The Beginning Fabrics Jungle Friends Small ...,In The Beginning Fabrics,100% Cotton,Lightweight (4.19 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/07895...
69,Sesame Street Digital Characters Packed Multi,Sesame Workshop,100% Cotton,Lightweight (4.19 oz/yd²),Quilting Cotton,https://images.fabric.com/images/279/279/06605...
70,Flannel Stars Yellow,A.E. Nathan,100% Cotton,44'',Blankets & Throws,https://images.fabric.com/images/279/279/03147...
